### Эмбеддинги в NLP

В этом задании мы научимся работать с моделями для построения векторных представлений текста. Загрузим и потестируем предобученные модели, посмотрим, какие свойства и функции у них есть, обучим свою модель и с помощью нее улучшим качество кластеризации(и/или классификации) текстов.

Перед тем как приступать к выполнению задания не забудьте выполнить команду pip install -r <path_to_rep_requirements.txt> для того, чтобы в вашем виртуальном окружении были установлены правильные версии python пакетов.

In [ ]:
import string
import warnings
import logging
import itertools

import nltk
import gensim
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import MiniBatchKMeans

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

In [ ]:
%pylab inline

plt.style.use('default')
figsize(12, 9)

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

mpl.rcParams['font.family'] = 'Ubuntu'

plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)
 
font = {'family': 'Verdana',
        'weight': 'normal'}
mpl.rc('font', **font)

Скачиваем rucorpora 15 с [диска](https://yadi.sk/d/fLRMFhm03Pbs98).
Либо выбираем предобученную модель с https://rusvectores.org/ru/models/. Выбор нужно обосновать.

In [ ]:
path_to_ruscorpora = 'ruscorpora.model.bin'

#### Загрузим обученную на ruscorpora модель word2vec

In [ ]:
model_word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    path_to_ruscorpora,
    binary=True
)
model_word2vec.wv.vocab

#### Посмотрим, как с ней обращаться

Можно посмотреть наиболее похожие на конкретное слово слова. Попробуйте свойства векторов word2vec: и подумайте какие слова нужно послать на вход, чтобы получить на выходе слова "школа",  "машинное", "обучение".

In [ ]:
model_word2vec.most_similar(positive=['корабль'],
                            negative=['парус'],
                            topn=3)

In [ ]:
model_word2vec.most_similar(positive=['корабль', 'летать'],
                            negative=['плавать'],
                            topn=3)

In [ ]:
model_word2vec.most_similar(positive=['шахтер'],
                            negative=['грязь'],
                            topn=3)

In [ ]:
model_word2vec.most_similar(positive=['учение', 'тьма'],
                            negative=['свет'],
                            topn=3)

Еще есть функция, которая выводит лишнее слово в строке. Попробуйте придумать пример, с которым word2vec не справится.

In [ ]:
model_word2vec.doesnt_match("коньяк компот водка пиво".split())

Функция, которая выводит сходство между словами.  С ее помощью можно понимать значение слова нефть.

In [ ]:
print(model_word2vec.similarity('нефть', 'газ'))
print(model_word2vec.similarity('нефть', 'вода'))
print(model_word2vec.similarity('нефть', 'водка'))
print(model_word2vec.similarity('нефть', 'духи'))
print(model_word2vec.similarity('нефть', 'компот'))

Или смотреть, где в России больше нефти.

In [ ]:
print(model_word2vec.similarity('нефть', 'чечня'))
print(model_word2vec.similarity('нефть', 'якутск'))
print(model_word2vec.similarity('нефть', 'москва'))
print(model_word2vec.similarity('нефть', 'саратов'))
print(model_word2vec.similarity('нефть', 'сибирь'))

### Пункт 0
Допишите своих интересных примеров, характеризующих word2vec, попробуйте обосновать полученные результаты.

In [ ]:
<Ваш код>

#### Теперь давайте обучим собственную модель
В качестве обучающего корпуса возьмем новости по 20 темам.

In [ ]:
train_all = fetch_20newsgroups(subset='train')
print(train_all.target_names, " - 20 возможных тем")

dataset = fetch_20newsgroups(
    subset='train', 
    categories=['comp.sys.mac.hardware', 'soc.religion.christian', 'rec.sport.hockey'])

dataset.data[0].split()

### Пункт 1

Предобработаем эти новости. Выкинем цифры, знаки пунктуации, переведем в нижний регистр, разобьем на слова.

Нормализуйте слова, например с помощью модуля nltk (или используйте любой другой способ на ваш выбор).

In [ ]:
def normalize(raw_text):
    preprocessed_text = <Ваш код>
    return preprocessed_text

In [ ]:
data_normalized = [normalize(news) for news in tqdm(dataset.data)]

Теперь обучающий корпус готов.

In [ ]:
sentences = data_normalized
model = gensim.models.Word2Vec(sentences, min_count=1)

In [ ]:
model = gensim.models.Word2Vec(iter=1)  # Инициализируем модель.
model.build_vocab(sentences)  # Строим словарь.
model.train(sentences, total_examples=model.corpus_count, epochs=20)  # Тренируем модель.

Нарисуем слова из первой новости в новом векторном пространстве.

In [ ]:
# Берем слова из первой новости, достаем соостветствующие векторы,
# выбрасываем слова, для которых векторов нет. 
# Подумайте - как так могло получится, что нет векторов?
labels = []
embeddings = []
data_to_tsne = [item for sublist in data_normalized[:20] for item in sublist]

for x in list(set(data_to_tsne)):
    try:
        embeddings.append(model[x])
        labels.append(x)
    except KeyError:
        continue

In [ ]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

    plt.savefig(filename)

### Пункт 2
Переведите многомерные векторы в двумерные (можно использовать TSNE, PCA, другие методы сокращения размерности), выберите часть слов для отрисовки. Как параметры и методы отрисовки влияют на полученный график? Как это можно обьяснить?

In [ ]:
def embeddings_2d(embeddings):
    return <Ваш код>

In [ ]:
num_words_to_draw = 1000
low_dim_vectors = embeddings_2d(embeddings)

In [ ]:
# Рисуем только первые 1000 слов.
labels_to_draw = [labels[i] for i in range(num_words_to_draw)]
a = plot_with_labels(low_dim_vectors, labels_to_draw)

### Пункт 3
Проделайте аналогичную отрисовку и сжатие векторов для предобученной модели (для этого скачайте бинарник по этой [ссылке](https://code.google.com/archive/p/word2vec/) (файл GoogleNews-vectors-negative300) по аналогии с тем, как мы работали с русским корпусом. Сравните результаты.

In [ ]:
<Ваш код>

### Попробуем с помощью нашей модели улучшить качество кластеризации текстов
Если вы не знаете, что такое кластеризация - почитать об этом можно по [ссылке](http://scikit-learn.org/stable/modules/clustering.html).

Если вам все равно кажется, что это сложно, и вы чувствуете себя более уверено в задаче классификации - вместо задачи кластеризации можно рассматривать задачу классификации - тогда вместо функции quality используйте roc-auc из библиотеки scikit-learn, все остальные шаги предобработки и измерения признаков аналогичны.

Измерять качество кластеризации будем следующим образом - измерим количество правильных попаданий категории в кластеры.

In [ ]:
def quality(preds, target):
    permutations = list(itertools.permutations([0, 1, 2]))
    scores = []
    for a, b, c in permutations:
        mapping = {2 : a, 1: b, 0: c}
        mapped_preds = [mapping[pred] for pred in preds]
        scores.append(float(sum(mapped_preds != target)) / len(target))
    return 1 - min(scores)

### Пункт 4
Кластеризуйте датасет с заданием признаков с помощью one hot encoding - посмотрите на результат.

In [ ]:
<Ваш код>

### Пункт 5
Далее кластеризуем тексты на векторах, полученных из модели, которую мы сами обучили.

Напишем функцию, которая делает из предложения вектор - усредняет векторы всех слов в этом предложении. Если слова нет в словаре, то вместо вектора этого слова используется нулевой вектор.

In [ ]:
def average_feature_vector(words, model):
    average_feature_vector = < Ваш код >
    return

Применим функцию к нашим данным (уже нормализованным функцией normalize).

In [ ]:
data_normalized_vectors = [average_feature_vector(sent, model)
                           for sent in data_normalized]

Сожмем массив признаков с помощью TSNE до нескольких компонент (попробуйте поварьировать число компонент).

In [ ]:
low_dim_vectors = < Ваш код >

Примените алгоритм кластеризации (например, K-means).

In [ ]:
preds = MiniBatchKMeans(n_clusters=3).fit_predict(low_dim_vectors)
quality(preds, dataset.target)

### Пункт 6

Кластеризуйте тексты на векторах, полученных из предобученной модели. Какой результат получился лучше? Почему?

In [ ]:
<Ваш код>

### Пункт 7

Попробуйте получить аналогичные векторы и провести исследование на них с помощью fastq - [неплохое введение](https://www.analyticsvidhya.com/blog/2017/07/word-representations-text-classification-using-fasttext-nlp-facebook/).

In [ ]:
<Ваш код>

### Пункт 8
С помощью библиотеки pytorch-pretrained-bert попробуйте получить векторы для ваших текстов ([инструкция](https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/extract_features.py)) и провести исследование.

In [ ]:
<Ваш код>

### Правила сдачи и критерии оценки:

##### Как и куда сдавать:

К блокноту нужно приложить краткий текстовый отчет в pdf, в котором вы описываете основные моменты: какие шаги выполняли, какое качество получали, а также какие графики рисовали и что это значит.

Оба файла (ipynb блокнот с названием hw07_<имя>_<фамилия>.ipynb и pdf файл-отчет с названанием hw07_<имя>_<фамилия>_report.pdf) нужно загрузить через [гугл-форму](https://forms.gle/JbSQ6toQHQQc3CcF9).

##### Обязательная часть (70% баллов)

1) Пункт 1 - Пункт 7 (по 10% баллов за пункт).

##### Продвинутая часть (30 % баллов)

1) Пункт 8.

2) Любые самые смелые идеи-эксперименты по тому, как можно улучшить качество моделей (пофильтровать какие-то части речи, редкие слова, использовать FastText).

3) Интересные, необычные, просто хорошо сформулированные выводы о вашей работе в отчете.

При проверке отчета будет обращаться внимание на анализ и оригинальность подходов.